### Convert Vocals to Phonk Style (Experimental)

#### 1st Try with Output as Female

In [1]:
from pydub import AudioSegment
from pydub.effects import speedup

def convert_to_phonk(file_path):
    audio = AudioSegment.from_mp3(file_path)
    
    # Pitch shift (simple approach by changing frame rate)
    pitch_shifted = audio._spawn(audio.raw_data, overrides={
         "frame_rate": int(audio.frame_rate * 1.35555)  # Adjust this factor as needed
    }).set_frame_rate(audio.frame_rate)
    
    # Speed up slightly for phonk effect
    phonk_audio = speedup(pitch_shifted, playback_speed=1.111111)  # Adjust playback speed as needed
    
    # Export phonk-styled audio
    phonk_audio.export("phonk_vocal_female_tone.wav", format="wav")

convert_to_phonk("latest_input.wav")

#### 2nd Try with New Output Male

In [2]:
from pydub import AudioSegment
from pydub.effects import speedup

def convert_to_phonk(file_path):
    audio = AudioSegment.from_mp3(file_path)
    
    # Pitch shift (simple approach by changing frame rate)
    pitch_shifted = audio._spawn(audio.raw_data, overrides={
         "frame_rate": int(audio.frame_rate * 0.79)  # Adjust this factor as needed
    }).set_frame_rate(audio.frame_rate)
    
    # Speed up slightly for phonk effect
    phonk_audio = speedup(pitch_shifted, playback_speed=1.4)  # Adjust playback speed as needed
    
    # Export phonk-styled audio
    phonk_audio.export("phonk_vocal_male_tone.wav", format="wav")

convert_to_phonk("latest_input.wav")

### Adding the beat to the Male and Female Audio

In [4]:
import numpy as np
import librosa
import soundfile as sf
import audioread

def load_audio_with_fallback(file_path, sr=None):
    try:
        return librosa.load(file_path, sr=sr)
    except Exception as e:
        print(f"Failed to load with librosa due to: {e}. Falling back to audioread.")
        with audioread.audio_open(file_path) as f:
            data = np.hstack([np.frombuffer(chunk, np.int16) for chunk in f])
            if sr is not None:
                data = librosa.resample(data.astype(float), f.samplerate, sr)
            return data, sr

def mix_vocals_and_beat(vocals_filename, beat_filename, output_filename, beat_start_time=0):
    vocals, sr_vocals = load_audio_with_fallback(vocals_filename, sr=None)
    beat, sr_beat = load_audio_with_fallback(beat_filename, sr=sr_vocals)

    start_sample = int(beat_start_time * sr_beat)
    if start_sample < len(beat):
        beat = np.concatenate((np.zeros(start_sample), beat))
    else:
        raise ValueError("Beat start time exceeds the length of the beat track.")

    # Ensure both tracks are of the same length
    max_length = max(len(vocals), len(beat))
    if len(vocals) < max_length:
        vocals = np.concatenate((vocals, np.zeros(max_length - len(vocals))))
    if len(beat) < max_length:
        beat = np.concatenate((beat, np.zeros(max_length - len(beat))))

    mixed = vocals + beat
    mixed = mixed / np.max(np.abs(mixed))

    sf.write(output_filename, mixed, sr_vocals)

def main():
    vocals_file = r"phonk_vocal_female_tone.wav"
    beat_file = r"only_beat.wav"
    output_file = r"Phonk_Female_Final.wav"

    mix_vocals_and_beat(vocals_file, beat_file, output_file, beat_start_time=0)

if __name__ == "__main__":
    main()

In [69]:
import numpy as np
import librosa
import soundfile as sf
import audioread

def simple_delay_reverb(audio, sr, delay_ms=0, decay=0, repeats=0):
    delay_samples = int(sr * delay_ms / 1000)
    output = np.copy(audio)
    for _ in range(repeats):
        delayed = np.zeros_like(output)
        delayed[delay_samples:] = output[:-delay_samples]
        output += delayed * decay
    output = output / np.max(np.abs(output))
    return output

def load_audio_with_fallback(file_path, sr=None):
    try:
        return librosa.load(file_path, sr=sr)
    except Exception as e:
        print(f"Failed to load with librosa due to: {e}. Falling back to audioread.")
        with audioread.audio_open(file_path) as f:
            data = np.hstack([np.frombuffer(chunk, np.int16) for chunk in f])
            if sr is not None:
                data = librosa.resample(data.astype(float), f.samplerate, sr)
            return data, sr

def modify_vocals(vocals_filename, beat_filename, output_filename, volume_gain=2.0, beat_start_time=0.0, pitch_shift_semitones=0):
    vocals, sr_vocals = load_audio_with_fallback(vocals_filename, sr=None)
    beat, sr_beat = load_audio_with_fallback(beat_filename, sr=sr_vocals)

    if pitch_shift_semitones != 0:
        vocals = librosa.effects.pitch_shift(vocals, sr_vocals, pitch_shift_semitones)

    vocals = simple_delay_reverb(vocals, sr_vocals)

    start_sample = int(beat_start_time * sr_beat)
    if start_sample < len(beat):
        beat = beat[start_sample:]
    else:
        raise ValueError("Beat start time exceeds the length of the beat track.")

    vocals = vocals * volume_gain

    min_length = min(len(vocals), len(beat))
    vocals = vocals[:min_length]
    beat = beat[:min_length]

    mixed = vocals + beat
    mixed = mixed / np.max(np.abs(mixed))

    sf.write(output_filename, mixed, sr_vocals)

def main():
    vocals_file = r"phonk_vocal_male_tone.wav"
    beat_file = r"NewBeatYT.wav"
    output_file = r"zMalePhonk.wav"

    modify_vocals(vocals_file, beat_file, output_file, volume_gain=3.0, beat_start_time=14.25, pitch_shift_semitones=0)

if __name__ == "__main__":
    main()